# 1. Introduction to neural networks

## Minimal neural network

Let us start by building a minimal neural network. First, run the following cell for the necessary imports (their significance will become clear later). You'll need to install the python package Tensorflow first using the Python Packages manager.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

Next, we download the famous MNIST dataset, which comes with Tensorflow:

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

First, we do some basic preprocessing:

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Define the structure of a simple neural network:

In [4]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

Compile the model:

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train the model:

In [6]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2897 - accuracy: 0.9178 - val_loss: 0.1621 - val_accuracy: 0.9546
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1282 - accuracy: 0.9627 - val_loss: 0.1168 - val_accuracy: 0.9653
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0872 - accuracy: 0.9734 - val_loss: 0.1026 - val_accuracy: 0.9696
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0646 - accuracy: 0.9805 - val_loss: 0.0979 - val_accuracy: 0.9720
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0498 - accuracy: 0.9849 - val_loss: 0.0902 - val_accuracy: 0.9729


In [8]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 1s 2ms/step - loss: 0.0777 - accuracy: 0.9758
Test accuracy: 0.9757999777793884
